In [9]:
from knn import recommender_knn_all_courses
from tfidf import recommender_tfidf_all_courses
import importlib
import pandas as pd
from fuzzywuzzy import process

Take list of courses

In [10]:
file = pd.read_excel('shortcourses2566.xlsx')
courses = pd.DataFrame(file)['หลักสูตรอบรมระยะสั้น'].drop_duplicates()
courses = courses.sort_values().set_axis(range(0,len(courses)))
courses

0                     Cancer Epidemiology and Prevention
1      Coaching Skill : Crafting a New You สำหรับนักศ...
2      Coaching Skill : Crafting a New You สำหรับบุคล...
3      Development Studies & Social Research Speciali...
4                     Digital Transformation: e-Document
                             ...                        
163                             เวชศาสตร์ครอบครัวขั้นสูง
164    เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้น...
165    แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข  (...
166    แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (S...
167    “ฟูมฟักทักษะสำคัญในชีวิตและการทำงาน Skills4Lif...
Name: หลักสูตรอบรมระยะสั้น, Length: 168, dtype: object

Take output of TF-IDF recommender

In [11]:
tfidf_output = recommender_tfidf_all_courses('การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency)')
tfidf_output

,Course,Score
Index,,
0,Cancer Epidemiology and Prevention,0.0
1,Coaching Skill : Crafting a New You สำหรับนักศ...,0.0
2,Coaching Skill : Crafting a New You สำหรับบุคล...,0.0
3,Development Studies & Social Research Speciali...,0.0
4,Digital Transformation: e-Document,0.0
...,...,...
163,เวชศาสตร์ครอบครัวขั้นสูง,0.0
164,เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้น...,0.0
165,แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (...,0.0


Take output of KNN recommender

In [12]:
knn_output = recommender_knn_all_courses('การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency)')
knn_output

Selected movie: การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency) Index: 42
Searching for recommendations...


,Course,Score
Index,,
0,Cancer Epidemiology and Prevention,0.000000
1,Coaching Skill : Crafting a New You สำหรับนักศ...,0.000000
2,Coaching Skill : Crafting a New You สำหรับบุคล...,0.000000
3,Development Studies & Social Research Speciali...,0.017835
4,Digital Transformation: e-Document,0.000000
...,...,...
163,เวชศาสตร์ครอบครัวขั้นสูง,0.000000
164,เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้น...,0.000000
165,แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (...,0.000000


Hybrid Recommender System

In [13]:
def recommender_hybrid_all_courses(course_name):
    idx = process.extractOne(course_name, courses)[2]
    remain_courses = courses.drop(idx)
    knn_score = recommender_knn_all_courses(course_name).loc[:, 'Score']
    tfidf_score = recommender_tfidf_all_courses(course_name).loc[:, 'Score']
    hybrid_score = knn_score * 0.5 + tfidf_score * 0.5
    df = {
        'Course': remain_courses,
        'Score': hybrid_score,
    }
    hybrid_output = pd.DataFrame(data=df)
    return hybrid_output

In [14]:
recommender_hybrid_all_courses('การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency)')

Selected movie: การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency) Index: 42
Searching for recommendations...


,Course,Score
0,Cancer Epidemiology and Prevention,0.000000
1,Coaching Skill : Crafting a New You สำหรับนักศ...,0.000000
2,Coaching Skill : Crafting a New You สำหรับบุคล...,0.000000
3,Development Studies & Social Research Speciali...,0.008917
4,Digital Transformation: e-Document,0.000000
...,...,...
163,เวชศาสตร์ครอบครัวขั้นสูง,0.000000
164,เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้น...,0.000000
165,แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (...,0.000000
166,แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (S...,0.000000


In [17]:
def recommender_hybrid_by_user(user_name, n_recommendations):
    df = {
        'User': pd.Series(file['ชื่อ-นามสกุล (อังกฤษ)']),
        'Course': pd.Series(file['หลักสูตรอบรมระยะสั้น'])
    }

    user_course = pd.DataFrame(df)
    selected_user_name = user_course.loc[user_course['User'] == user_name]
    selected_courses = selected_user_name['Course']

    recommended_courses = [ recommender_hybrid_all_courses(x) for x in selected_courses]

    # pre dataframe
    df = pd.DataFrame({
        'Course': [],
        'Score': []
    }).rename_axis('Index')

    for x in recommended_courses:
        df = df._append(x)
    df =  df.sort_values('Score', ascending=False).drop_duplicates('Course')
    return df.head(n_recommendations)

In [16]:
recommender_hybrid_by_user('PORPHAING JANTIP', 10)

Selected movie: หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (Basic Principle of Diagnostic Radiology Imaging Instruments) Index: 131
Searching for recommendations...
Selected movie: การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเหตุ (Diagnostic Radiology of Non-Traumatic Emergency) Index: 43
Searching for recommendations...
Selected movie: การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency) Index: 42
Searching for recommendations...


,Course,Score
43,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.726190
42,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.726190
131,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.320202
152,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.265576
110,รังสีวิทยาวินิจฉัย,0.129466
140,หลักเศรษฐศาสตร์สาธารณสุข (Principle of Health ...,0.110981
133,หลักสูตรย่อยที่ 1 พื้นฐานการดูแลสุขภาพสัตว์เลี...,0.104536
130,หลักการบริหารงานสาธารณสุข (Principle of Public...,0.096543
32,การตรวจวิเคราะห์พื้นฐานทางพิษวิทยาและการตรวจวิ...,0.090358
154,เทคโนโลยีก๊าซชีวภาพ : หลักการการออกแบบ และการใ...,0.078547
